In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader
from picopic.plain_reader import PlainReader

In [3]:
##  configuration options
data_path = '../../'

timestamp=0
show_grid=False
use_cache=False
cmap='terrain'

if os.path.isfile(os.path.join(data_path, "metadata.json")):
    reader = PlainReader(path = data_path, use_cache=use_cache, verbose=True)
elif os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

cmp = 'position' # component
specie = 'electrons'
shape = 'rec'
# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = 1 # cfg.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]
image_interpolation = 'nearest'

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'

plot_name = r'$\mathbf{Something\enspace to\enspace Debug}$'

In [4]:
# define plot builder
plot = PlotBuilder(0, 0,
                   fig_color=reader.meta.figure_color,
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height,
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9)

In [5]:
# get data
data_r = data_z = data_phi = []

for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.shape == 'rec') and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
        if probe.component == cmp: 
            data_r = reader.get_frame("{}/{}".format(cmp, specie), size, "{}_0".format(frame))
            data_phi = reader.get_frame("{}/{}".format(cmp, specie), size, "{}_1".format(frame))
            data_z = reader.get_frame("{}/{}".format(cmp, specie), size, "{}_2".format(frame))

Loading data set position/electrons/rec:0-128_0-512/0_0...done
Loading data set position/electrons/rec:0-128_0-512/0_1...done
Loading data set position/electrons/rec:0-128_0-512/0_2...done


In [6]:
# add subplots
plot_p = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# plot_p.set_xlim([0,reader.meta.geometry_size[1]])
# plot_p.set_ylim([0,reader.meta.geometry_size[0]])
# add images
plot_p.plot(data_z, data_r, 'b,')

plot.show()